In [115]:
import html
import bs4
import requests
import feedparser
import os

feed_reuters = feedparser.parse('http://web.archive.org/web/20200613003232if_/http://feeds.reuters.com/Reuters/worldNews')

DIR = "./CA03"

soupified = []
file_names = []
for url in feed_reuters.entries:
    file_name = url.id.split("/")[-1] + ".html"
    html = requests.get(url.id)
    path = os.path.join(DIR, file_name)
    (f := open(path,"w+")).write(html.text)
    f.close()
    soup = bs4.BeautifulSoup(html.text, 'html.parser')
    soupified.append(soup)
    file_names.append(file_name)

In [185]:
titles = list(map(lambda a: a.select("h1")[0].text, soupified))

In [186]:
text = list(
    map(lambda soup: 
        ' '.join(
            map(lambda a: a.text,
                soup.select('p.ArticleBody-para-TD_9x')
            )),soupified))
#this finds all p tags with class ArticleBody-para-TD_9x

In [187]:
dates = list(
    map(lambda soup: 
        soup.find("meta",
            {"name":"analyticsAttributes.articleDate"}
        )["content"], soupified))
#this searchs for the meta tag with a specific name and grabes the content attribute from it

In [140]:
import pandas

In [141]:
df = pandas.DataFrame({"Title":titles,"Time":dates,"Content":text})
#make a dataframe

In [142]:
df

Title                  Time  \
0   Mexico City to begin gradual exit from lockdow...  2020-06-13T00:50:37Z   
1   Mexico reports record tally of 5,222 new coron...  2020-06-13T00:15:39Z   
2   Venezuela top court names new electoral counci...  2020-06-13T01:23:40Z   
3   One-fifth of Britain's coronavirus patients we...  2020-06-12T23:01:39Z   
4   France to lift border controls for EU travelle...  2020-06-12T22:57:35Z   
5   Brazil's COVID-19 deaths surge past UK, WHO sa...  2020-06-13T12:16:14Z   
6   Canada's Trudeau calls arrest video of indigen...  2020-06-12T22:19:29Z   
7   Egypt registers highest daily rise in coronavi...  2020-06-12T22:09:57Z   
8   Artists around the world pay tribute to George...  2020-06-12T22:07:53Z   
9   Brazil's COVID-19 death toll passes Britain, w...  2020-06-12T22:02:22Z   
10  Lebanon protesters burn roads, clash with secu...  2020-06-12T21:30:06Z   
11  'Stop buying social peace at our expense', Fre...  2020-06-12T21:28:56Z   
12  Soccer team mobs coffin of murdered Mexican te...  2020-06-12T21:14:18Z   
13  Chile's military moves through shadows to spot...  2020-06-12T21:02:53Z   
14  Somalia's Islamist group al Shabaab says sets ...  2020-06-12T20:38:31Z   
15  Congo's gold being smuggled out by the tonne, ...  2020-06-12T20:31:38Z   
16  Brazil drops police from human rights report a...  2020-06-12T20:27:40Z   
17  Slovenia to open borders for Italians, Montene...  2020-06-12T20:26:23Z   
18  Botswana reinstates strict coronavirus lockdow...  2020-06-12T20:25:01Z   
19  Coronavirus hitting the Americas hardest says ...  2020-06-12T20:15:19Z   

                                              Content  
0   MEXICO CITY (Reuters) - The mayor of Mexico Ci...  
1   MEXICO CITY (Reuters) - Mexico’s health minist...  
2   CARACAS (Reuters) - Venezuela’s supreme court ...  
3   (Reuters) - About one in five of Britain’s pat...  
4   PARIS (Reuters) - France will lift restriction...  
5   GENEVA/BRASILIA (Reuters) - Brazil’s COVID-19 ...  
6   OTTAWA/WINNIPEG, Manitoba (Reuters) - Prime Mi...  
7   CAIRO (Reuters) - Egypt on Friday confirmed 1,...  
8   (Reuters) - Protests against racism triggered ...  
9   BRASILIA (Reuters) - Brazil’s COVID-19 death t...  
10  BEIRUT (Reuters) - Lebanese protesters set fir...  
11  PARIS (Reuters) - French police marched in pro...  
12  VINCENTE CAMALOTE, Mexico (Reuters) - Mexican ...  
13  SANTIAGO (Reuters) - The armored vehicle rumbl...  
14  MOGADISHU (Reuters) - Somalia’s Islamist group...  
15  (Reuters) - Gold production in Democratic Repu...  
16  SÃO PAULO (Reuters) - Brazil has excluded comp...  
17  LJUBLJANA (Reuters) - Slovenia will open its b...  
18  GABORONE (Reuters) - Botswana brought back a s...  
19  ZURICH/GENEVA (Reuters) - The Americas are bea...

In [143]:
#time to cleanup and stuff


In [144]:
import html
import re
#copy paste of the code from the document
### Standard cleaning function
def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text) 
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [145]:
titles = list(map(clean,titles))
dates = list(map(clean,dates))
text = list(map(clean,text))
#quick little map function for applying the function over the stream

In [146]:
import textacy
import textacy.preprocessing as tprep
#yet another copy paste
if textacy.__version__ < '0.11':
    def normalize(text):
        text = tprep.normalize_hyphenated_words(text)
        text = tprep.normalize_quotation_marks(text)
        text = tprep.normalize_unicode(text)
        text = tprep.remove_accents(text)
        return text
else:
    # adjusted to textacy 0.11. Note, function names are changed
    def normalize(text):
        text = tprep.normalize.hyphenated_words(text)
        text = tprep.normalize.quotation_marks(text)
        text = tprep.normalize.unicode(text)
        text = tprep.remove.accents(text)
        return text

In [147]:
titles = list(map(normalize,titles))
dates = list(map(normalize,dates))
text = list(map(normalize,text))
#yet another map for normalize

In [148]:
import spacy
nlp = spacy.load('en_core_web_sm')
#import no need to explain

In [157]:
lemmas = list(map(lambda toks: 
            list(map(lambda tok: tok.lemma_ ,toks)),
            map(nlp,text)))

#this mpas the function for the spacy nlp function, such that its result will get a list of list of lemma tokens

In [150]:
import textacy

In [173]:
nouns = list(map(lambda doc: 
            list(map(lambda a: a.__repr__(),
                textacy.extract.matches.token_matches(doc,patterns=["POS:NOUN"]))),
            map(nlp,text)))
#this uses the matches.token_matches to return a set of tokens such that it matches the pattern, a noun

In [174]:
noun_phrases = list(map(lambda doc: 
            list(map(lambda a: a.__repr__(),
                textacy.extract.matches.token_matches(doc,patterns=["POS:ADJ POS:NOUN:+"]))),
            map(nlp,text)))

#this uses the matches.token_matches to return a set of tokens such that it matches the pattern, a adjective then a noun
#the reason for __repr__ is that it makes it so that it can be pcikled

In [175]:
type(noun_phrases[0][0])

str

In [176]:
named_entites = list(map(lambda a: list(map(lambda ent: (ent.text.__repr__(),ent.label_.__repr__()),a.ents)), map(nlp,text)))
#returns the named entites from the tokens

In [177]:
#i dunno if you want it be a pickled dict of 1d or 2d lists, so im going do both.
topickle = {"Lemma":lemmas,"Noun":nouns,"Noun Phrase":noun_phrases,"Entity Pairs": named_entites}

In [178]:
onpickle = {key: sum(topickle[key],[]) for key in topickle.keys()}

In [179]:
import pickle

In [184]:
def pickle_obj(obj, path):
    f = open(path,"wb")
    pickle.dump(obj, f)
    f.close()
#quick and dirty function to pickle objects for use
    
pickle_obj(topickle,"./CA03/pickles/2dpickle.pkl")
pickle_obj(onpickle,"./CA03/pickles/1dpickle.pkl")
pickle_obj(df,"./CA03/pickles/dataframe.pkl")

{'Lemma': ['MEXICO',
  'CITY',
  '(',
  'Reuters',
  ')',
  'the',
  'mayor',
  'of',
  'Mexico',
  'City',
  'say',
  'on',
  'Friday',
  'the',
  'mexican',
  'capital',
  'will',
  'next',
  'week',
  'lift',
  'restriction',
  'on',
  'car',
  'traffic',
  'and',
  'public',
  'transport',
  ',',
  'and',
  'allow',
  '340,000',
  'factory',
  'work',
  'to',
  'get',
  'back',
  'to',
  'work',
  ',',
  'even',
  'though',
  'new',
  'case',
  'of',
  'coronavirus',
  'be',
  'still',
  'rise',
  '.',
  'Mexico',
  'City',
  'and',
  'the',
  'adjacent',
  'urban',
  'sprawl',
  'be',
  'home',
  'to',
  'more',
  'than',
  '21',
  'million',
  'people',
  ',',
  'and',
  'the',
  'region',
  'account',
  'for',
  'more',
  'than',
  '40',
  '%',
  'of',
  'some',
  '139,000',
  'confirm',
  'case',
  'of',
  'coronavirus',
  'in',
  'the',
  'country',
  '.',
  'on',
  'Friday',
  'Mexico',
  'report',
  'a',
  'record',
  'of',
  'more',
  'than',
  '5,000',
  'new',
  'case',
 